In [2]:
!pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 40.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [9]:
pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sagemaker 
bucket = sagemaker.Session().default_bucket()

prefix = 'sagemaker/DEMO-xgboost-fraud'

#Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()  #provides acces to sagemaker to other aws resources

In [2]:
print(role) 

arn:aws:iam::344208508792:role/service-role/AmazonSageMaker-ExecutionRole-20221130T145708


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt       #for charts and visualization    
from IPython.display import Image     #for displaying images in the notebook
from IPython.display import display   #for displaying outputs in the notebook
from time import gmtime, strftime     #for labeling sagemaker models, endpoint, etc.
import sys                            #for writing outputs to notebook
import math
import os 
import json
import sagemaker
import zipfile 

In [6]:
sagemaker.__version__

'2.116.0'

# Data

lets start by downloading and reading in the credit card fraud data set

In [9]:
!wget https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip

with zipfile.ZipFile('creditcardfraud.zip', 'r') as zip_ref: 
    zip_ref.extractall('.')

--2022-12-01 19:14:28--  https://s3-us-west-2.amazonaws.com/sagemaker-e2e-solutions/fraud-detection/creditcardfraud.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.248.144, 52.92.161.40, 52.92.176.48, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.248.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69155632 (66M) [application/zip]
Saving to: ‘creditcardfraud.zip.1’

100%[======================================>] 69,155,632  6.60MB/s   in 11s    

2022-12-01 19:14:39 (5.92 MB/s) - ‘creditcardfraud.zip.1’ saved [69155632/69155632]



now lets read this into a Pandas data frame and take a look.

In [12]:
data = pd.read_csv('./creditcard.csv') 
print(data.columns)
data[['Time', 'V1', 'V2', 'V27', 'V28', 'Amount', 'Class']].describe()
data.head(10) 

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


La coumna class corresponde a si es una transaccion fraudulenta o no. Es posible observar que la mayoria de la data es no fraudulenta con solo 492 (.173%) de la informacion correspondiendo a ejemplos de fraude. 

In [13]:
nonfrauds, frauds  = data.groupby('Class').size()
print('Number of frauds: ', frauds)
print('Number of non-frauds: ', nonfrauds)
print('Percentage of fradulent data', 100.*frauds/(frauds + nonfrauds))

Number of frauds:  492
Number of non-frauds:  284315
Percentage of fradulent data 0.1727485630620034


In [15]:
feature_columns = data.columns[:-1]
label_column = data.columns[-1]

features = data[feature_columns].values.astype('float32')
labels = data[label_column].values.astype('float32') 

XGBOOST necesita que la variable target este en la primera columna, por lo tanto, es necesario restructurar el dataframe.

In [16]:
model_data = data
model_data.head()
model_data = pd.concat([model_data['Class'], model_data.drop(['Class'], axis = 1)], axis = 1)
model_data.head() 

,Class,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,0,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,0,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,0,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


Split data en train y validacion. 

In [29]:
train_data, validation_data, test_data = np.split(model_data.sample(frac =1, random_state = 1729),
                                                 [int(0.7*len(model_data)), int(0.9 * len(model_data))])

train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index =False) 


Uploading data to S3

In [30]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

s3_train_data = 's3://{}/{}/train/train.csv'.format(bucket, prefix)
s3_validation_data = 's3://{}/{}/validation/validation.csv'.format(bucket, prefix)
print('Uploading training data location: {}'.format(s3_train_data))
print('Uploading validation data location: {}'.format(s3_validation_data))

output_location = 's3://{}/{}/output'.format(bucket, prefix)
print('Training artifacts will be uploading to: {}'.format(output_location))

Uploading training data location: s3://sagemaker-us-east-1-344208508792/sagemaker/DEMO-xgboost-fraud/train/train.csv
Uploading validation data location: s3://sagemaker-us-east-1-344208508792/sagemaker/DEMO-xgboost-fraud/validation/validation.csv
Training artifacts will be uploading to: s3://sagemaker-us-east-1-344208508792/sagemaker/DEMO-xgboost-fraud/output


# Training

Firts we´ll need to specify the location of the XGBOOST algorith containers. To specify the linear learner algorith, we use a utility function to obtain it´s URI.

xgboost is an extremely popular, open source package for gradient trees. It is computationally powerfull, fully featured, and has been successfully used in many machine learning competitions. 

Firts we´ll need to specify the ECR container location for amazone sagemaker´s implementation og XGBOOST.

In [31]:
container = sagemaker.image_uris.retrieve(region = boto3.Session().region_name, framework = 'xgboost', version='latest')

Then, because we´re training with the CSV file format, we´ll create s3_input s that our training function can use as a pointer to the file in S3, which also specify that the content type is CSV. 

In [32]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/train'.format(bucket, prefix), content_type = 'csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data = 's3://{}/{}/validation'.format(bucket, prefix), content_type = 'csv')

first we´ll need to specify training parameters to the estimator. this includes:

1. The xgboost algorithm container
2. The IAM role to use
3. Training instance type and count
4. S3 location for output  data
5. Algorithm hyperparameters

And the a .fit() function which specifies:

1. S3 location for output data. In this case we have both a training and validation set which are passed in. 

In [36]:
sess = sagemaker.Session() 

xgb  =sagemaker.estimator.Estimator(container,
                                   role,
                                   instance_count = 1,
                                   instance_type = 'ml.m4.xlarge',
                                   output_path = 's3://{}/{}/output'.format(bucket, prefix),
                                   sagameker_Session = sess)

xgb.set_hyperparameters(max_depth = 5,
                       eta = 0.2,
                       gamma = 4,
                       min_child_weight = 6,
                       subsample = 0.8,
                       silent = 0,
                       objective = 'binary:logistic',
                       num_round = 100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

2022-12-01 20:33:37 Starting - Starting the training job...ProfilerReport-1669926817: InProgress
...
2022-12-01 20:34:20 Starting - Preparing the instances for training.........
2022-12-01 20:36:01 Downloading - Downloading input data...
2022-12-01 20:36:37 Training - Downloading the training image......
2022-12-01 20:37:23 Training - Training image download completed. Training in progress.Arguments: train
[2022-12-01:20:37:26:INFO] Running standalone xgboost training.
[2022-12-01:20:37:26:INFO] File size need to be processed in the node: 129.45mb. Available memory size in the node: 8822.2mb
[2022-12-01:20:37:26:INFO] Determined delimiter of CSV input is ','
[20:37:26] S3DistributionType set as FullyReplicated
[20:37:27] 199364x30 matrix with 5980920 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-12-01:20:37:27:INFO] Determined delimiter of CSV input is ','
[20:37:27] S3DistributionType set as FullyReplicated
[20:37:27] 56962x30 matrix with 170

# Hosting

Now that we have´ve trained the XGBOOST algorithm on our data, let´s deploy a model that´s hosted behind a real time endpoint

In [38]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

-------!

# Evaluation

There are many ways to compare the performance of a machine learning model, but let´s start by symply comparing actual to predict values. In this case, we´re simply predicting wheter its a fraudulent transaction (1) or not (0), which produces a simple confusion matrix.

First, we´ll need to determine how pass data into and receive data from our endpoint. Our data is currently stored as NumPy arrays in memory of our notebook instance. To send it in an HTTP POST request, we´ll serialize it as a CSV string and then decode the resulting CSV. 

Note: for inference with csv format, SageMaker XGboost requires that the data does not include the target variable. 

In [58]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

Now, we´ll use a simple function to:

1. loops over out trest dataset
2. split it into mini-batches of rows
3. convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
4. retrieve mini-batch predictions by invoking the XGboost endpoint
5. collect predictions and convert from the CSV output our model provides into a NumPy array. 

Now, we´ll check out confusion matrix to see how well we predicted versus actuals.

In [116]:
def predict(data, rows=1000):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
    return np.fromstring(predictions[1:], sep=',')

predictions=predict(test_data.to_numpy()[:,1:])  
predictions.shape 

(28481,)

In [123]:
predictions

array([3.95067781e-03, 1.48141489e-03, 1.37213661e-04, ...,
       2.20631318e-05, 4.21777331e-05, 8.69680662e-06])

In [117]:
pd.crosstab(index = test_data.iloc[:, 0], columns = np.round(predictions), rownames = ['actual'], colnames = ['predictions'])   

predictions,0.0,1.0
actual,,
0,28426,1
1,11,43


In [124]:
import sklearn.model_selection 

cutoff=0.5
print(sklearn.metrics.confusion_matrix(test_data.iloc[:, 0], np.where(np.round(predictions) > cutoff, 1, 0)))
print(sklearn.metrics.classification_report(test_data.iloc[:, 0], np.where(np.round(predictions)> cutoff, 1, 0)))

[[28426     1]
 [   11    43]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28427
           1       0.98      0.80      0.88        54

    accuracy                           1.00     28481
   macro avg       0.99      0.90      0.94     28481
weighted avg       1.00      1.00      1.00     28481

